In [ ]:
import pandas as pd

data_custom_ratios = pd.read_csv('/Users/keshavsaraogi/data/e-commerce/ecommerce_customer_data_custom_ratios.csv')
data_large = pd.read_csv('/Users/keshavsaraogi/data/e-commerce/ecommerce_customer_data_large.csv')

In [ ]:
print("Custom Ratios Shape:\n", data_custom_ratios.shape)
print("Custom Ratios Head:\n", data_custom_ratios.head())
print("Custom Ratios Info:\n", data_custom_ratios.info())

In [ ]:
print("Large Dataset Shape:", data_large.shape)
print("Large Dataset Head:\n", data_large.head())
print("Custom Ratios Info:\n", data_large.info())

In [ ]:
print("Missing Values in Custom Ratios Dataset:\n", data_custom_ratios.isnull().sum())
print("Missing Values in Large Dataset:\n", data_large.isnull().sum())

In [ ]:
# the empty returns signifies no returns, thus I am going ahead with 0 to ensure there are no returns made

data_custom_ratios['Returns'] = data_custom_ratios['Returns'].fillna(0)
data_large['Returns'] = data_large['Returns'].fillna(0)